In [1]:
import sympy as sym
import numpy as np
import math
import scipy.linalg as la 
import matplotlib.pyplot as plt
import networkx as nx
from numpy.linalg import inv
import scipy.stats as stats

In [2]:
t = sym.Symbol('t', real=True)
x = sym.Symbol('x', real=True)
pi = np.pi

In [13]:
#Use for random number generator use (run this cell before running any random results)
np.random.seed(10) 

In [3]:
def matrix_exp(H):
    """Function that takes in a matrix and returns the matrix exponential"""
    eigvals, eigvecs = la.eig(H)
    diag_vals = []
    for i in range(len(eigvals)):
        diag_vals.append(sym.exp(1j*eigvals[i]*t))
    W = eigvecs
    X = inv(W)
    D = sym.diag(*diag_vals)
    M = sym.Matrix(W)*D*sym.Matrix(X)
    return M 

In [4]:
def line(N):
    """Function that generates the adjacency matrix for a line of N nodes"""
    a = np.zeros((1, N))[0]
    b = np.ones((1, N-1))[0]
    H = np.diag(a, 0) + np.diag(b, -1) + np.diag(b, 1)
    return H 

def loop(N):
    """Function that generates the adjacency matrix for a loop of N nodes"""
    a = np.zeros((1, N))[0]
    b = np.ones((1, N-1))[0]
    H = np.diag(a, 0) + np.diag(b, -1) + np.diag(b, 1)
    H[0][N-1] = 1
    H[N-1][0] = 1
    return H 

def k_connected(N):
    """Function that generates the adjacency matrix for a k-connected graph of N nodes"""
    H = np.ones((N,N))
    for i in range(N):
        H[i][i] = 0
    return H 

def plot_graph(graph,N):
    """A function that takes in a graph function and the size to returns a plot of the graph"""
    H = graph(N)
    A = nx.from_numpy_matrix(np.array(H))  
    nx.draw_circular(A, with_labels=True)
    plt.title(f'{graph.__name__} Graph')
    plt.show()

In [5]:
def probability_distribution(H, start_node, final_node, time_array):
    """A function that takes in an adjacency matrix, the start and finish nodes and the time array and generates the probability distribtution and analytical probability function"""
    M = matrix_exp(H)
    Y = M*sym.Matrix(start_node)
    Z = sym.Matrix(final_node).dot(Y)

    F_p = sym.Abs(sym.simplify(Z))**2
    P = []
    for i in range(len(time_array)):
        p = F_p.evalf(subs={t:time_array[i]})
        P.append(sym.Abs(p))
      
    return P, F_p

def fisher_info(F_p, time_array):
    """A function that takes in an analytical probability function and time array and returns the fisher information with time as an array"""
    F = []
    df = sym.diff(F_p, t)
    f = sym.simplify((df**2)/(F_p))
    for i in range (len(time)):
        fi = f.evalf(subs={t:time_array[i]})
        F.append(sym.Abs(fi)) 
    return F

In [6]:
def produce_data(P,n,set_val):
    """A function that takes an array of probability functions, the number of data points produced and a set value as parameter and returns the randomly sampled data array"""
    N = len(P)
    Pe = np.zeros(N)
    for i in range(N):
        Pe[i] = sym.re(P[i].evalf(subs={t:set_val}))
    nodes = np.arange(N)
    M = np.zeros(N)
    for i in range(n):
        d = np.random.choice(nodes, p=Pe)
        M[d] += 1
    return M 

def MLF(P,M):
    """A function that takes an aray of probability functions and sampled data and returns the analytical maximum liklihood function"""
    mlf = 1
    for i in range(len(P)):
        mlf = mlf*(P[i]**int(M[i]))
    return mlf

In [7]:
def get_max(F, X):
    """A function that takes in a y (F) and x arrays and returns the values of any local maximum"""
    max_vals = []
    for i in range(1, len(X)-1):
        f_low = sym.Abs(F.subs(t,X[i-1]))
        f = sym.Abs(F.subs(t,X[i]))
        f_high = sym.Abs(F.subs(t,X[i+1]))
        if f_low < f and f_high < f:
            max_vals.append(X[i])
    return max_vals

def get_max_2(F, X):
    """A function that takes in a y (F) and x arrays and returns one maximum values"""
    K = 0
    J = 0
    for j in range(len(X)):
        k = sym.re(F.evalf(subs={t:X[j]}))
        if k > K:
            K = k
            J = j
    return X[J]

In [8]:
def produce_nodes(N):
    """A function that takes in a size N and returns an array of node vectors"""
    node = np.zeros(N)
    nodes = []
    for i in range(N):
        node[i] = 1
        nodes.append(node)
        node = np.zeros(N)
    return nodes

In [9]:
def produce_probs_fish(H, node_i):
    """A function that takes in an adjacency matrix and starting node and produces the probability and fisher arrays with time (undefined) and returns the analytical probabilty function array"""
    N = len(H)
    nodes = produce_nodes(N)
    probs = []
    pf = []
    fish = []
    for j in range(N):
        probi, pfi = probability_distribution(H, nodes[node_i], nodes[j], time)
        fi = fisher_info(pfi, time)
        probs.append(probi)
        pf.append(pfi)
        fish.append(fi)
    return probs, pf, fish

C = ['mediumspringgreen','navy','lightblue','forestgreen','lime','teal']

def fock_plot(H, node_i):
    """A function that takes in an adjacency matrix and starting node and produces the plot of the probability distribution and fisher information with time"""
    N = len(H)
    probs, pf, fish = produce_probs_fish(H, node_i)
    fig, ax = plt.subplots(2, figsize=(10,6))
    fig.suptitle(f"Fock State on {N} Nodes")

    ax[0].set(ylabel="Probability")

    thickness = np.linspace(1,3,N)

    for i in range(N):
        ax[0].plot(time, probs[i], label = f'Node {i}', c=C[i], linewidth=thickness[N-i-1])
        ax[1].plot(time, fish[i], label = f'Node {i}', c=C[i], linewidth=thickness[N-i-1])

    ax[1].set(xlabel='Time (s)', ylabel='Fisher Information')
    ax[1].plot(time, np.sum(fish, axis=0), label ="Total", c='darkviolet')
    ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.show()

In [10]:
def find_nearest(array, value):
    """A function that takes in an array and a value and returns the index of the nearest in the array to the value"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

In [11]:
def MLE_plot_single(H, node_i, set_val):
    """A function that takes in an adjacency matrix,starting node and parameter value and plots a single maximum liklihood function using standard N=50 for producing data"""
    probs, pf, fish = produce_probs_fish(H, node_i)
    M = produce_data(pf, 50, set_val)
    mlf = MLF(pf, M)

    mlf_array = []
    for y in time:
        mlf_array.append(mlf.evalf(subs={t:y}))
    plt.plot(time,mlf_array)

    max_vals = get_max(mlf,time)
    plt.scatter(max_vals,np.zeros(len(max_vals)), marker='x', c='red')
    plt.show()

In [12]:
def MLE_repeats(H,node_i,n,repeats,set_val,period):
    """A function that takes in an adjacency matrix and starting node, N(n) - number of repeats for data production, repeats - number of times experiment is repeated and set parameter value. The function also takes in a 'period' - this needs to be judged from the probability graph. This returns the mean, variance, time array and array of y arrays for the maximum liklihood functions"""
    probs, pf, fish = produce_probs_fish(H,node_i)
    time_1 = time[:find_nearest(time,period/2)+1]

    data = []
    multiple_mlf = []
    for j in range(repeats):
        M = produce_data(pf, n, set_val)
        mlf = MLF(pf, M)
        mlf_array = []
        for y in time_1:
            mlf_array.append(mlf.evalf(subs={t:y}))
        multiple_mlf.append(mlf_array)
        max_vals = get_max_2(mlf,time_1)
        data.append(max_vals)
    mean = np.mean(data)
    var = np.var(data)
    return mean,var,time_1,multiple_mlf

def MLE_plot_mult(mean,var,time,multiple_mlf):
    """This takes in the outputs of MLE_repeats and plots the multiple maximum liklihood functions and the normal distribution associated"""
    fig, ax = plt.subplots(2, figsize=(8,8))
    for j in range(len(multiple_mlf)):
        ax[0].plot(time, multiple_mlf[j])
    ax[0].set(ylabel="Liklihood")
    ax[0].set_title('Maximum Likelihood Estimation Function')
    ax[1].set_title('Normal PDF')
    ax[1].set(xlabel='Time (s)')
    ax[1].plot(time, stats.norm.pdf(time_1, mean, math.sqrt(var)))
    ax[1].scatter(mean,0, marker='x', c='red')
    plt.show()